# Analyzing the social security reform with reinforced learning

Tässä tehdään laskelmat artikkelia varten. Käytössä on gym-ympäristö _unemployment-v1_ , johon on toteutettu yksityiskohtaisesti sosiaaliturvaa eri tiloissa.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.10
  
# restart kernel after running pip's

In [2]:
#import sys
#print(sys.path)
#sys.path.append('/usr/local/python3.7/site-packages')

Then load all modules and set parameters for simulations.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [4]:
# parameters for the simulation
pop_size=100_000 # size of the population to be simulated
size=10_000_000 # number of timesteps in phase 1 training (callback not used)
batch=8 # size of minibatch in phase 1 as number of episodes
deterministic=True # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
rlmodel='leaky_acktr' # use ACKTR algorithm with leaky ReLU
year=2018

# Baseline

The baseline model for Finnish social security

In [8]:
baseline_model='baseline_model'
baseline_results='baseline_results'
baseline_start='v3_malli_baseline3test_nomort'
baseline_LCmodel=Lifecycle(env='unemployment-v3')

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 18
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_kesto500 500
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True
perustulo False

sigma_reduction True
plotdebug False

additional_tyel_premium 0.0
scale_tyel_accrual False include_ove False

version 3


In [ ]:
baseline_LCmodel.run_results(steps1=size,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch,rlmodel=rlmodel,
                save=baseline_model,cont=True,start_from=baseline_start,results=baseline_results,
                learning_rate=0.25,log_interval=1000)

In [ ]:
baseline_LCmodel.render(load=baseline_results,figname='v3_')

# Increasing the retirement age

Above the retirement age was 63.5 years. Here we increase it to 65 years.

In [5]:
retirement_model='retirement_LCmodel'
retirement_results='retirement_results'

Now, set up the Lifecycle model with a higher lowest retirement age

In [6]:
ret_LC=Lifecycle(env='unemployment-v3',min_retirementage=65)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 18
max_age 70
min_retirementage 65
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_kesto500 500
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True
perustulo False

sigma_reduction True
plotdebug False

additional_tyel_premium 0.0
scale_tyel_accrual False include_ove False

version 3


And then run the simulation.

In [9]:
ret_LC.run_results(steps1=size,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch,
                save=retirement_results,cont=True,start_from=baseline_start,results=retirement_model,
                rlmodel=rlmodel,learning_rate=0.25,log_interval=1000)

train...
phase 1
batch 1 learning rate 0.0625 scaled 0.0625 n_cpu 8




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...






---------------------------------
| explained_variance | 0.995    |
| fps                | 281      |
| nupdates           | 1        |
| policy_entropy     | 1.34     |
| policy_loss        | -0.128   |
| total_timesteps    | 1680     |
| value_loss         | 0.0664   |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps                | 1272     |
| nupdates           | 1000     |
| policy_entropy     | 1.38     |
| policy_loss        | -0.00604 |
| total_timesteps    | 1680000  |
| value_loss         | 0.0251   |
---------------------------------
---------------------------------
| explained_variance | 0.

Population:   0%|          | 0/10000 [00:00<?, ? p/s]

render method shows the results.

In [ ]:
ret_LC.render()

In [ ]:
ret_LC.compare_with(baseline_LCmodel,label1='retirement 65',label2='retirement 63.5',figname='retage')